In [8]:
!pip install yfinance
!pip install pytickersymbols

In [9]:
import yfinance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pytickersymbols import PyTickerSymbols
from google.colab import files

stock_data = PyTickerSymbols()
nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index()

horarios = ['09:30:00','09:45:00', '10:00:00', '10:15:00', '10:30:00', '10:45:00', '11:30:00']
#period = [10]   #días (para Bollinger)
period= [10,15,20,10,15,20,10,15,20,10,15,20]
#multiplier = [2.5]   #Desviaciones estándar (para Bollingers)
multiplier = [1,1,1,1.5,1.5,1.5,2,2,2,2.5,2.5,2.5]
limit = [10]*12   #Limit of days to hold stock
#limit = [5,10,15,5,10,15,5,10,15,5,10,15]

cash = 5000
comision = 3

In [10]:
df_totales = pd.DataFrame(['Total Trades', 'GREENS', 'PL(G)', 'PL/Trade(G)', 'Max(G)', 
                           'REDS', 'PL(R)','PL/Trade(R)', 'Max(R)', 'Total PL'], 
                          index=['Total Trades', 'GREENS', 'PL(G)', 'PL/Trade(G)', 'Max(G)', 
                           'REDS', 'PL(R)','PL/Trade(R)', 'Max(R)', 'Total PL'])
df_totales2 = df_totales.drop(columns=0)

In [11]:
#d = pd.DataFrame(index=df.index)
#d_cierres = pd.DataFrame(index=range(1, limit+1))

In [12]:
iteracion = pd.DataFrame(['Periodo (bb)', 'Desv Estandar', 'Límite días'], index=['Periodo (bb)', 'Std', 'Límite días']).drop(columns=0)

In [13]:
totales = pd.DataFrame(['Total de trades:', 'Total de PL:', 'PL total/trade:', 'Porcentaje de Win Primer día:',
                        'Porcentaje de Win otros días:', 'Porcentaje de losers:','PL/Green trade:','PL/Red trade:'], 
                       index= ['Total de trades:', 'Total de PL:', 'PL total/trade:', 'Porcentaje de Win Primer día:',
                        'Porcentaje de Win otros días:', 'Porcentaje de losers:','PL/Green trade:','PL/Red trade:']).drop(columns=0)

In [14]:
%%time

for k in range(len(period)):
  df_totales = pd.DataFrame(['Total Trades', 'GREENS', 'PL(G)', 'PL/Trade(G)', 'Max(G)', 
                           'REDS', 'PL(R)','PL/Trade(R)', 'Max(R)', 'Total PL'], 
                          index=['Total Trades', 'GREENS', 'PL(G)', 'PL/Trade(G)', 'Max(G)', 
                           'REDS', 'PL(R)','PL/Trade(R)', 'Max(R)', 'Total PL'])
  df_totales2 = pd.DataFrame.copy(df_totales.drop(columns=0))
  #d = pd.DataFrame.copy(pd.DataFrame(index=df.index))
  d_cierres = pd.DataFrame.copy(pd.DataFrame(index=range(1, limit[k]+1)))

  for j in range(len(nasdaq_100.symbol)):

    df = yfinance.download(nasdaq_100.symbol[j],period='60d', interval='30m')

    df['UpperBand'] = df['Close'].rolling(period[k]).mean() + df['Close'].rolling(period[k]).std() * multiplier[k]
    #df['LowerBand'] = df['Close'].rolling(period[k]).mean() - df['Close'].rolling(period[k]).std() * multiplier[k]

    df2 = df[['Open', 'Close', 'UpperBand']]

    df2['Outside'] = np.nan
    for i in range(len(df2)):
      if df2['Close'][i] > df2['UpperBand'][i]:
        df2['Outside'][i] = True
      else:
        df2['Outside'][i] = False

    df2['Trigger'] = np.nan
    for i in range(1, len(df2)):
      #CONDICIONES DE TRIGGER: 1) FUERA DEL BOLLINGER EL DIA PREVIO Y 2) EL DIA ACTUAL, Y 3) SE ENCUENTRA EN LOS HORARIOS PARA ALCANZAR DAY TRADING
      if df2['Outside'][i-1] == True and df2['Open'][i] > df2['UpperBand'][i-1] and df2.index[i].strftime('%H:%M:%S') in horarios:
        df2['Trigger'][i] = 1.0
  
    #df2['After Day'] = np.nan
    df2['PL'] = np.nan

    pl = 0
    closed = 0
    pl_list = []
    closed_list = []
    for i in range(1, len(df2)):
      if (df2['Trigger'][i] == 1.0 or pl < 0):
        pl = pl + (((df2['Open'][i] - df2['Close'][i])/df2['Open'][i]) * cash)
        closed = closed + 1
        #CONDICIONES PARA CERRAR POSICION:.................
        if pl > 0 or closed == limit[k] or df2.index[i] == df2.index[-1]:
          df2['PL'][i] = pl - comision
          pl_list.append(pl - comision)
          closed_list.append(closed)
          closed = 0
          pl = 0

    df2['Green'] = np.nan
    df2['Red'] = np.nan
    for i in range(1, len(df2)):
      if df2['PL'][i] > 0:
        df2['Green'][i] = df2['PL'][i]
      elif df2['PL'][i] < 0:
        df2['Red'][i] = df2['PL'][i]
  
    df_totales2.insert(j, nasdaq_100.symbol[j], [df2['PL'].count(), df2['Green'].count(), df2['Green'].sum(),
                      df2['Green'].mean(), df2['Green'].max(), 
                      df2['Red'].count(), df2['Red'].sum(), df2['Red'].mean(), 
                      df2['Red'].max(), df2['PL'].sum()])
  
    pl_list = df2['PL'].to_list()

    # while len(pl_list) < len(d):
    #   pl_list.append(0)
      #closed_list.append(0)
  
    dfclosed = pd.DataFrame(closed_list) #Se pasa la lista a df

    if len(closed_list) > 0:
      inter = pd.DataFrame(dfclosed[0].value_counts()) #Se sacan el número de ocurrencias
      inter = inter.sort_index().reindex(range(1,limit[k]+1), fill_value=0) #Se ordena y se llenan los espacios
      inter = inter[0].to_list()
      #d.insert(j, nasdaq_100.symbol[j], pl_list)
    d_cierres.insert(j, nasdaq_100.symbol[j], inter)
  
  d_cierres['sum'] = d_cierres.sum(axis=1)
  iteracion.insert(k,k, [period[k], multiplier[k], limit[k]])
  df_totales3 = df_totales2.transpose()
  totales.insert(k, k, [df_totales3['Total Trades'].sum(), df_totales3['Total PL'].sum(), df_totales3['Total PL'].sum()/df_totales3['Total Trades'].sum(),
                 d_cierres['sum'][1]/df_totales3['Total Trades'].sum(), (df_totales3['GREENS'].sum()-d_cierres['sum'][1])/df_totales3['Total Trades'].sum(),
                 df_totales3['REDS'].sum()/df_totales3['Total Trades'].sum(), df_totales3['PL/Trade(G)'].mean(), df_totales3['PL/Trade(R)'].mean()])

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/magics/execution.py:1193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code, glob, local_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/magics/execution.py:1193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code, glob, local_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

[*********************100%***********************]  1 of 1 completed


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/magics/execution.py:1193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code, glob, local_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/magics/execution.py:1193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code, glob, local_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

[*********************100%***********************]  1 of 1 completed
CPU times: user 4min 10s, sys: 5.09 s, total: 4min 15s
Wall time: 7min 34s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/magics/execution.py:1193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code, glob, local_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [15]:
df_totales2

,AAPL,CSCO,INTC,MSFT,WBA,ATVI,ADBE,ALXN,GOOGL,AMZN,AMGN,ADI,AMAT,ADSK,ADP,BIDU,BIIB,CERN,CHTR,CHKP,CTSH,CMCSA,COST,CSX,DLTR,EBAY,EA,FB,FAST,FISV,GILD,ILMN,INCY,INTU,ISRG,JD,KHC,LRCX,MAR,MXIM,...,TSLA,TXN,VRSK,VRTX,XLNX,BKNG,EXC,AMD,ALGN,AEP,ANSS,AVGO,CDNS,CTAS,CPRT,FOXA,IDXX,KLAC,SNPS,VRSN,ASML,CDW,DXCM,PEP,XEL,TEAM,DOCU,KDP,LULU,MRVL,MTCH,MELI,MRNA,OKTA,PTON,PDD,SGEN,SPLK,WDAY,ZM
Total Trades,10.000000,13.000000,9.000000,8.000000,11.000000,10.000000,6.000000,8.000000,7.000000,6.000000,4.000000,15.000000,10.000000,5.000000,6.000000,13.000000,9.000000,12.000000,7.000000,7.000000,4.000000,10.000000,5.000000,2.000000,9.000000,11.000000,9.000000,5.000000,5.000000,4.000000,9.000000,15.000000,6.000000,6.000000,10.000000,9.000000,7.000000,11.000000,7.000000,13.000000,...,7.000000,8.000000,5.000000,6.000000,11.000000,4.000000,5.000000,10.000000,10.000000,6.000000,9.000000,6.000000,7.000000,2.000000,7.000000,9.000000,6.000000,10.000000,8.000000,5.000000,15.000000,9.000000,8.000000,5.000000,4.000000,6.000000,14.000000,7.000000,8.000000,13.000000,7.000000,8.000000,9.000000,12.000000,9.000000,9.000000,8.000000,4.000000,11.000000,6.000000
GREENS,9.000000,8.000000,6.000000,6.000000,9.000000,6.000000,3.000000,6.000000,5.000000,3.000000,3.000000,10.000000,8.000000,2.000000,5.000000,9.000000,5.000000,11.000000,5.000000,3.000000,4.000000,6.000000,3.000000,1.000000,8.000000,6.000000,6.000000,3.000000,4.000000,1.000000,3.000000,11.000000,2.000000,4.000000,9.000000,8.000000,6.000000,9.000000,3.000000,8.000000,...,4.000000,6.000000,2.000000,4.000000,10.000000,2.000000,4.000000,9.000000,9.000000,4.000000,6.000000,5.000000,5.000000,1.000000,6.000000,5.000000,6.000000,7.000000,4.000000,2.000000,7.000000,7.000000,6.000000,3.000000,4.000000,3.000000,11.000000,7.000000,6.000000,11.000000,6.000000,7.000000,7.000000,10.000000,8.000000,9.000000,6.000000,3.000000,7.000000,5.000000
PL(G),252.087918,60.525449,121.380281,76.280977,262.190566,88.872078,64.419826,78.439460,132.246705,64.233752,58.021355,193.354564,304.756768,118.254049,147.259379,482.157319,160.850081,157.247929,56.552910,59.851421,31.231259,60.455732,36.306546,30.455791,176.235347,205.346560,81.166786,43.642003,79.119122,56.336112,18.964977,395.257080,40.066934,55.584817,79.484090,274.125707,49.891888,263.004845,23.402843,96.378269,...,121.453733,121.603777,8.512284,207.908521,382.545341,56.707912,48.165987,347.071398,200.270841,112.481775,181.259340,73.760658,124.198422,8.434974,112.568307,71.238955,196.168531,216.619112,127.573672,23.912052,120.496010,229.312421,181.947874,12.662624,33.130140,104.489186,384.559035,214.882842,115.947961,288.217304,143.572932,255.724970,413.318966,317.553670,374.118482,829.031621,220.952250,159.988015,86.915772,181.462488
PL/Trade(G),28.009769,7.565681,20.230047,12.713496,29.132285,14.812013,21.473275,13.073243,26.449341,21.411251,19.340452,19.335456,38.094596,59.127024,29.451876,53.573035,32.170016,14.295266,11.310582,19.950474,7.807815,10.075955,12.102182,30.455791,22.029418,34.224427,13.527798,14.547334,19.779780,56.336112,6.321659,35.932462,20.033467,13.896204,8.831566,34.265713,8.315315,29.222761,7.800948,12.047284,...,30.363433,20.267296,4.256142,51.977130,38.254534,28.353956,12.041497,38.563489,22.252316,28.120444,30.209890,14.752132,24.839684,8.434974,18.761385,14.247791,32.694755,30.945587,31.893418,11.956026,17.213716,32.758917,30.324646,4.220875,8.282535,34.829729,34.959912,30.697549,19.324660,26.201573,23.928822,36.532139,59.045567,31.755367,46.764810,92.114625,36.825375,53.329338,12.416539,36.292498
Max(G),104.466926,18.302129,49.718431,44.204677,77.994572,28.653871,40.332200,42.434125,49.048089,30.171983,27.865602,84.172921,71.131565,113.378525,96.164786,125.068780,62.322691,39.437660,23.263393,51.386666,20.440592,22.643449,22.622997,30.455791,38.736815,50.386670,27.908612,20.235862,64.662916,56.336112,7.798908,138.696765,38.209703,39.831961,35.540675,86.739906,22.054948,72.789409,11.127343,31.843174,...,53.358704,58.023356,7.123352,75.043399,116.6

In [16]:
#df_totales2.loc['Max(G)'].sort_values().dropna().tail(6).sum()

In [17]:
#df2.loc[(df2['Trigger']==1) | (df2['PL']<0) | (df2['PL']>0)]

In [18]:
iteracion

,0,1,2,3,4,5,6,7,8,9,10,11
Periodo (bb),10,15,20,10.0,15.0,20.0,10,15,20,10.0,15.0,20.0
Std,1,1,1,1.5,1.5,1.5,2,2,2,2.5,2.5,2.5
Límite días,10,10,10,10.0,10.0,10.0,10,10,10,10.0,10.0,10.0


In [19]:
totales

,0,1,2,3,4,5,6,7,8,9,10,11
Total de trades:,4369.000000,4255.000000,3947.000000,3148.000000,2988.000000,2586.000000,1801.000000,1799.000000,1552.000000,666.000000,795.000000,812.000000
Total de PL:,-15525.009873,-12449.708995,-6771.291069,-6573.378383,-6940.067006,-3769.850547,-1333.711393,-2764.898854,-222.512186,532.930537,175.546689,772.860932
PL total/trade:,-3.553447,-2.925901,-1.715554,-2.088113,-2.322646,-1.457792,-0.740539,-1.536909,-0.143371,0.800196,0.220813,0.951799
Porcentaje de Win Primer día:,0.491646,0.494007,0.503167,0.489517,0.488286,0.494973,0.494725,0.476376,0.492912,0.509009,0.476730,0.491379
Porcentaje de Win otros días:,0.166857,0.170623,0.172283,0.180750,0.177041,0.168987,0.189339,0.192885,0.191366,0.195195,0.210063,0.214286
Porcentaje de losers:,0.341497,0.335370,0.324550,0.329733,0.334672,0.336040,0.315936,0.330739,0.315722,0.295796,0.313208,0.294335
PL/Green trade:,22.641010,21.419691,22.649814,22.674801,21.246473,23.147525,23.094926,22.087261,23.721540,25.390216,25.156412,25.227097
PL/Red trade:,-56.510441,-53.971719,-55.505296,-55.225312,-52.375030,-53.354858,-54.935250,-51.721667,-54.324699,-59.127156,-55.721733,-60.870163


In [20]:
# print('Total de trades:', df_totales3['Total Trades'].sum())
# print('Total de PL:',df_totales3['Total PL'].sum())
# print('PL total/trade:',df_totales3['Total PL'].sum()/df_totales3['Total Trades'].sum())
# print('-'*50)
# print('Porcentaje de Win Primer día:',d_cierres['sum'][1]/df_totales3['Total Trades'].sum())
# print('Porcentaje de Win otros días:',(df_totales3['GREENS'].sum()-d_cierres['sum'][1])/df_totales3['Total Trades'].sum())
# print('Porcentaje de losers:',df_totales3['REDS'].sum()/df_totales3['Total Trades'].sum())
# print('-'*50)
# print('PL/Green trade:',df_totales3['PL/Trade(G)'].mean())
# print('PL/Red trade:',df_totales3['PL/Trade(R)'].mean())

In [21]:
#DATAFRAME DE LOS DIAS QUE TOMÓ UNA POSICION PARA CERRAR, POR ACCIÓN
d_cierres['sum'] = d_cierres.sum(axis=1)
print(d_cierres['sum'].sum())
d_cierres

1624


,AAPL,CSCO,INTC,MSFT,WBA,ATVI,ADBE,ALXN,GOOGL,AMZN,AMGN,ADI,AMAT,ADSK,ADP,BIDU,BIIB,CERN,CHTR,CHKP,CTSH,CMCSA,COST,CSX,DLTR,EBAY,EA,FB,FAST,FISV,GILD,ILMN,INCY,INTU,ISRG,JD,KHC,LRCX,MAR,MXIM,...,TXN,VRSK,VRTX,XLNX,BKNG,EXC,AMD,ALGN,AEP,ANSS,AVGO,CDNS,CTAS,CPRT,FOXA,IDXX,KLAC,SNPS,VRSN,ASML,CDW,DXCM,PEP,XEL,TEAM,DOCU,KDP,LULU,MRVL,MTCH,MELI,MRNA,OKTA,PTON,PDD,SGEN,SPLK,WDAY,ZM,sum
1,8,6,5,3,6,7,2,4,4,2,1,9,4,3,4,6,1,7,3,2,1,7,0,1,7,3,3,2,2,2,3,8,1,3,4,6,2,5,3,5,...,3,2,3,7,1,3,8,7,4,2,4,2,2,4,3,4,3,3,3,4,7,5,1,0,3,9,5,6,8,4,5,7,9,5,7,5,3,2,3,798
2,1,0,1,1,1,0,1,1,0,1,2,1,4,0,2,1,1,2,1,0,1,0,1,0,1,3,3,1,0,0,1,0,2,1,2,1,2,2,1,1,...,1,1,0,2,1,1,1,0,0,1,0,1,0,1,1,2,1,2,0,4,1,3,1,2,0,0,1,0,2,1,2,0,0,1,0,0,0,1,2,212
3,0,1,0,3,0,1,0,0,1,0,0,0,0,0,0,1,1,2,1,1,1,0,0,1,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,1,...,2,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,0,0,3,0,90
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,46
5,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,28
6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,20
7,0,1,0,0,0,0,0,2,0,0,0,1,0,0,0,1,2,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,32
8,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,22
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,16
10,1,2,3,1,2,2,2,1,2,3,0,3,2,2,0,4,3,1,0,3,0,3,2,0,1,4,2,1,0,1,5,4,3,2,1,1,1,3,2,4,...,2,2,3,2,2,0,1,1,0,2,1,2,0,1,4,0,3,3,1,7,0,0,2,0,3,2,0,2,2,0,1,2,1,2,0,2,1,3,1,360


In [22]:
#DATAFRAME CON EL P/L RESULTANTE AL CERRAR LA POSICIÓN, POR DÍA ANALIZADO Y POR ACCIÓN
#d['sum'] = d.sum(axis=1).cumsum()
#d

In [23]:
#P/L ACUMULADO
# plt.figure(figsize=[24,8])
# d['sum'].plot()

In [24]:
#ANÁLISIS FINAL POR ACCION
df_totales3 = df_totales2.transpose()
#df_totales3

In [25]:
print('Total de trades:', df_totales3['Total Trades'].sum())
print('Total de PL:',df_totales3['Total PL'].sum())
print('PL total/trade:',df_totales3['Total PL'].sum()/df_totales3['Total Trades'].sum())
print('-'*50)
print('Porcentaje de Win Primer día:',d_cierres['sum'][1]/df_totales3['Total Trades'].sum())
print('Porcentaje de Win otros días:',(df_totales3['GREENS'].sum()-d_cierres['sum'][1])/df_totales3['Total Trades'].sum())
print('Porcentaje de losers:',df_totales3['REDS'].sum()/df_totales3['Total Trades'].sum())
print('-'*50)
print('PL/Green trade:',df_totales3['PL/Trade(G)'].mean())
print('PL/Red trade:',df_totales3['PL/Trade(R)'].mean())

Total de trades: 812.0
Total de PL: 772.8609321874551
PL total/trade: 0.9517991775707575
--------------------------------------------------
Porcentaje de Win Primer día: 0.9827586206896551
Porcentaje de Win otros días: -0.2770935960591133
Porcentaje de losers: 0.29433497536945813
--------------------------------------------------
PL/Green trade: 25.227096814206767
PL/Red trade: -60.87016258257552
